# Calculate dipole matrix elements

In [1]:
import logging

import numpy as np

from numerov.angular import calc_angular_matrix_element
from numerov.radial import calc_radial_matrix_element
from numerov.rydberg import RydbergState
from numerov.units import BaseQuantities, ureg

logging.basicConfig(level=logging.INFO, format="%(levelname)s %(filename)s: %(message)s")
logging.getLogger("numerov").setLevel(logging.WARNING)

In [2]:
operator = "p"
k_radial, k_angular = 1, 1

n = 60
l = 2
j = l - 0.5
m = j - 1
qn_i = [n, l, j, m]
qn_f = [n, l + 1, j + 1, m]

q = qn_f[-1] - qn_i[-1]

print(qn_i)
print(qn_f)

[60, 2, 1.5, 0.5]
[60, 3, 2.5, 0.5]


In [3]:
state_i = RydbergState("Rb", *qn_i)
state_f = RydbergState("Rb", *qn_f)

radial = calc_radial_matrix_element(state_i, state_f, k_radial)
radial *= BaseQuantities["RADIAL_MATRIX_ELEMENT"] ** k_radial
angular = calc_angular_matrix_element(state_i, state_f, "p", k_angular, q)
dipole = radial * angular * ureg.Quantity(1, "e")

print(f"Numerov radial matrix element: {radial}")
print(f"Numerov angular matrix element: {angular}")
print(f"Numerov dipole matrix element: {dipole}")

Numerov radial matrix element: 137.78307575236317 bohr
Numerov angular matrix element: 0.4898979485566356
Numerov dipole matrix element: 67.49964615690624 bohr * elementary_charge


In [4]:
if int(np.__version__.split(".")[0]) < 2:
    import os

    from pairinteraction import pireal as pi

    os.makedirs("./.pairinteraction_cache/", exist_ok=True)
    cache = pi.MatrixElementCache("./.pairinteraction_cache/")
    cache.setMethod(pi.NUMEROV)

    pi_state_i = pi.StateOne("Rb", *qn_i)
    pi_state_f = pi.StateOne("Rb", *qn_f)

    radial_pi = cache.getRadial(pi_state_f, pi_state_i, k_radial)
    radial_pi *= ureg.Quantity(1, "micrometer").to_base_units()
    print(f"Pairinteraction Numerov radial: {radial_pi}")

    dipole_pi = cache.getElectricMultipole(pi_state_i, pi_state_f, k_radial, k_angular)
    pi_unit_to_au = (ureg.Quantity(1, "cm/V") * ureg.Quantity(1, "GHz").to("J", "spectroscopy")).to("e * a_0")
    dipole_pi = dipole_pi * pi_unit_to_au
    print(f"Pairinteraction Numerov dipole: {dipole_pi}")

else:
    print(f"pairinteraction not available with numpy >= 2.0, but numpy {np.__version__} is installed.")

Pairinteraction Numerov radial: 137.7952691138138 bohr
Pairinteraction Numerov dipole: 67.50562268207688 bohr * elementary_charge


In [5]:
print(f"Radial: Pairinteraction radial / Numerov radial {radial_pi / radial}")
print(f"Dipole: Pairinteraction dipole / Numerov dipole {dipole_pi / dipole}")

Radial: Pairinteraction radial / Numerov radial 1.000088496801106 dimensionless
Dipole: Pairinteraction dipole / Numerov dipole 1.0000885415777847 dimensionless
